# REGRESSION

In this notebook, `ElasticNet` Regression is done which combines L1 and L2 penalties

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import style

import os
import sys
import inspect
from Data_Preprocessing import *
import pickle
import glob 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import LeaveOneOut
from tqdm import tqdm 


# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

## 1.1 Getting Scores
---

In [ ]:
#Getting parent directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

#getting data
mat_path = parentdir+'/Feature_Extraction/DATA/FC_Stroke/FCMatrixImage_131subj.mat'
lang_path = parentdir+'/Feature_Extraction/DATA/FC_Stroke/language_score.xlsx'

#we take both the fc matrices and the languages scores even if we dont use the FC matrices---here we only care about the scores
fc_3d, language_score, ID = get_arrays(mat_path, lang_path, True)

## 1.2 Importing features
---

Given the method, we will import the features and weights. 

In [ ]:
#i.e now importing PCA features: choose the directory according to import the correct feat
feats = open(parentdir+"/Results/Results_Extractors/PCA_RESULTS/FEATURES_PCA.pkl", "rb")
features_PCA = pickle.load(feats)

weights = open(parentdir+"/Results/Results_Extractors/PCA_RESULTS/WEIGHTS_PCA.pkl", "rb")
W_PCA = pickle.load(weights)

## 1.3 Removing empty scores values

---
Since now we care about the availability of the scores, we will now remove those features that has no correspondent scores. Furthremore, the features and the scores are standarize.

In [ ]:
from Data_Preparation import data_cleansing
features, scores = data_cleansing(language_score, features_PCA)

In [ ]:
#standarizing everything
scores_scaled = (scores - scores.mean())/scores.std()

for key in features.keys():
    for i in range(len(features[key].T)):
        features[key].T[i] = (features[key].T[i] -  np.mean(features[key], 1))/ np.std(features[key], 1)
   

## 1.4 ElasticNet Regression

---

Following the paper, we will perfomed elasticnet regression. Notice that the difference between the matlab function and the python one is in the definition of the alphas and lambdas (they are the opposite). 
Now, the optimal parameter are obtained by means of leave one out. IN particular we will use the `LeaveOneOut` function available from sklearn.

In [ ]:
#parameter search
alphas = np.logspace(-5, 5, 100)
lambdas =  [0.001, 0.25, 0.5, 0.75,1]

In [ ]:
def elasticnet(xtr, ytr, xte, yte, alph,l ):
    modelo = ElasticNet(
            l1_ratio        = l,
            alpha          = alph,
            normalize       = False,
            max_iter = 100000
         )
    modelo.fit(xtr, ytr)
    ypred = modelo.predict(xte)
    coef = modelo.coef_
    return ypred, coef

In [ ]:
def opt_lambda(Xtr, ytr, Xte, yte, ALPHA, LAMBDAS):
    
    nlamb = len(LAMBDAS)
    y_lamb = np.zeros (nlamb) 
    mse_lamb = np.zeros (nlamb) 
    coeficientes = np.zeros([ nlamb,Xtr.shape[1]])

    for idx_l, lamd in enumerate(LAMBDAS):
        
        y_pred, coef_ = elasticnet(Xtr, ytr, Xte, yte, ALPHA, lamd)
        coeficientes[idx_l,:] = coef_
        y_lamb[idx_l] = y_pred
        mse_lamb[idx_l] = ((yte - y_pred)**2)
        
    return y_lamb, mse_lamb, coeficientes


In [ ]:
def opt_alpha(X_train_, y_train_, X_test_, y_test_, alp, lambdas_):
    nalpha = len(alp)
    nlamb = len(lambdas_)
    y_alp = np.zeros([nlamb, nalpha])
    mse_alp = np.zeros([nlamb, nalpha])
    Coeficient = np.zeros([nalpha, nlamb, X_train_.shape[1]])


    for i_al, alfas in enumerate(alp):
        y_lamb_, mse_lamb_,coeficientes_  = opt_lambda(X_train_, y_train_, X_test_, y_test_, alfas, lambdas_)
        y_alp[:,i_al] = y_lamb_
        mse_alp[:,i_al] =mse_lamb_
        Coeficient[i_al,:,:] = coeficientes_
    return y_alp, mse_alp, Coeficient

In [ ]:
def loocv(X, y, alphas, lambdas):
    nalpha = len(alphas)
    nlamb = len(lambdas)
    npca = len(features)
    n_subj = len(features['n10'])
    
    y_hat = np.zeros([nlamb, nalpha, npca,n_subj])
    MSE = np.zeros([nlamb, nalpha, npca])
    
    loo = LeaveOneOut()
    loo.get_n_splits(X)
    coefi = []

    idx = 0
    for key in tqdm(features.keys()):
        X = features[key]
        y = scores
        
        subj_idx = 0
        sum_coef = np.zeros([nalpha,nlamb,X.shape[1]])

        for train_index, test_index in loo.split(X):
        
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

        
            y_alp_, mse_alp_, coefic_ = opt_alpha(X_train, y_train, X_test, y_test, alphas , lambdas)
            
            y_hat[:,:,idx,subj_idx] = y_alp_
            
            MSE[:,:,idx] += mse_alp_
            sum_coef +=coefic_
            subj_idx +=1
        print(MSE[:,:,idx].min()/n_subj)  
        idx +=1
        coefi.append(sum_coef)
    return y_hat, MSE, coefi

#### Performing loocv

In [ ]:
y_hat, MSE, coefi = loocv(features, scores_scaled,  alphas, lambdas)

## 1.5 Analysis of the model

---

In [ ]:
##getting best values
#diving the mean squared error with the total number of subject
MSE /= len(features['n10'])

#getting best (meaning mininum MSE) value
l_indeces = []
a_indeces = []
f_indeces = []

for lamb_idx in range(len(MSE)):
    for alph_idx in range(len(MSE[0])):
        for n_features in range(len(MSE[0][0])):
            if (MSE.min() == MSE[lamb_idx,alph_idx,n_features]):
                best_lambda = lambdas[lamb_idx]
                best_alpha = alphas[alph_idx]
                best_feat = list(features.keys())[n_features]
                print("Best lambda", lambdas[lamb_idx], 
                      ", Best Alpha", alphas[alph_idx], 
                      ", Best N_component", list(features.keys())[n_features], 
                     ", MSE", MSE.min())
                l_indeces.append(lamb_idx)
                a_indeces.append(alph_idx)
                f_indeces.append(n_features)
                
#best y and best coeficients
best_y =  y_hat[l_indeces[0], a_indeces[0], f_indeces[0], :]
best_coef = coefi[f_indeces[0]][a_indeces[0]][l_indeces[0]]

In [ ]:
#Saving 
MODELS  = ['AUGMENTATED','CONV AE', 'ICA', 'One linear layer AE', 'Non linear layer AE', 'PCA', 'TRANSFER']
model = MODELS[5]
np.save('RESULTS/Y_pred_'+str(model), y_hat)
np.save('RESULTS/MSE_'+str(model), MSE)
np.save('RESULTS/Best_Coeficients_'+str(model), best_coef)

In [ ]:
#get R^2 for best model
y = scores_scaled 
sse = sum((best_y - y)**2)
sst = sum((y - np.mean(y))**2)
R_2 = 1 - sse / sst;
print("R squared:", R_2)

#get BIC value  for best model
ns = len(y);
nzc = np.sum(best_coef != 0)
bic = ns + ns * np.log(2*np.pi) + ns * np.log(sse/ns) + np.log(ns) * nzc;
print("BIC:", bic)

In [ ]:
#saving best values
pca_fold = pd.DataFrame({'$R^2$': R_2,
              'MSE': MSE.min(),
              'BIC': bic, 
              'Best Lambda': best_lambda,
              'Best Alpha': best_alpha,
              'Best Fold': best_feat,
               'NZ': nzc},  index=[str(model)])

pca_fold.to_csv('RESULTS/'+str(model)+'.csv')